In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!nvidia-smi

Sat Nov 30 16:44:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              33W /  70W |    705MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [22]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import random
import math
import pickle
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from glob import glob

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchvision.models import efficientnet_v2_s

try:
  import optuna
except:
  !pip install optuna
  import optuna


In [23]:
%cd /content/drive/MyDrive/SWE3052
sys.path.append('/content/drive/MyDrive/SWE3052')

/content/drive/MyDrive/SWE3052


### 1. 하이퍼파라미터 설정 및 데이터셋

In [24]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 200
STEP_EPOCHS = 10
OPTUNA_NUM_EPOCHS=10
OPTUNA_NUM_TRIALS=10
STEP_EPOCHS = 10
BATCH_SIZE = 16
NUM_CLASSES = 30 # len(train_dataset.classes)
LR = 0.001
MOMENTUM = 0.9

In [25]:
class WasteDataset(Dataset):
    def __init__(self, root_dir, split, transform=None):
        self.root_dir = root_dir
        self.transform = transform  # Callable transform (e.g., transforms.Compose)
        self.classes = sorted(os.listdir(root_dir))
        self.image_paths = []
        self.labels = []

        for i, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for subfolder in ['default', 'real_world']:
                subfolder_dir = os.path.join(class_dir, subfolder)

                # 경로가 없는 경우 무시
                if not os.path.exists(subfolder_dir):
                    continue

                image_names = os.listdir(subfolder_dir)
                random.shuffle(image_names)

                if split == 'train':
                    image_names = image_names[:int(0.6 * len(image_names))]
                elif split == 'val':
                    image_names = image_names[int(0.6 * len(image_names)):int(0.8 * len(image_names))]
                else:  # split == 'test'
                    image_names = image_names[int(0.8 * len(image_names)):]

                for image_name in image_names:
                    self.image_paths.append(os.path.join(subfolder_dir, image_name))
                    self.labels.append(i)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)  # Correct usage of transform

        return image, label

In [26]:
# 데이터 증강 및 전처리 정의
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),         # 랜덤 크롭
        transforms.RandomHorizontalFlip(p=0.5),   # 랜덤 수평 플립
        transforms.RandomRotation(15),           # 랜덤 회전
        transforms.ToTensor(),                   # 텐서 변환
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),                  # 이미지 크기 조정
        transforms.CenterCrop(224),             # 중심 크롭
        transforms.ToTensor(),                  # 텐서 변환
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),                  # 이미지 크기 조정
        transforms.CenterCrop(224),             # 중심 크롭
        transforms.ToTensor(),                  # 텐서 변환
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
    ])
}

In [27]:
# 데이터셋 로드
root_dir = "./data/images/images"
train_dataset = WasteDataset(root_dir, split='train', transform=transform['train'])
val_dataset = WasteDataset(root_dir, split='val', transform=transform['val'])
test_dataset = WasteDataset(root_dir, split='test', transform=transform['test'])

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 9000
Validation dataset size: 3000
Test dataset size: 3000


### 2. ImageNet으로 사전학습한 모델 불러오기

In [28]:
# 사전 학습 모델 로드
model = efficientnet_v2_s(weights="EfficientNet_V2_S_Weights.IMAGENET1K_V1")

# 사전 학습된 파라미터 freeze
for param in model.features.parameters():
    param.requires_grad = False

# classifier
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(model.classifier[1].in_features, NUM_CLASSES)
)

model = model.to(DEVICE)

In [29]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Trainable Parameters in %: {round((trainable_params/total_params)*100, 2)}%")

# 모델 파라미터 개수 확인
count_parameters(model)

Total parameters: 20,215,918
Trainable parameters: 38,430
Trainable Parameters in %: 0.19%


In [11]:
# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)

# 학습 속도 스케줄러
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

### 3. 하이퍼파라미터 튜닝

In [12]:
def objective(trial):
    # 하이퍼파라미터
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)

    # 옵티마이저, 스케줄러
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    best_val_loss = float('inf')
    for epoch in range(OPTUNA_NUM_EPOCHS):
        model.train()
        train_loss = 0.0

        print(f'[Epoch {epoch+1}/{OPTUNA_NUM_EPOCHS}]')

        # 학습
        for images, labels in tqdm(train_loader, desc=f"[Train]"):
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

        # 검증
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"[Valid]"):
                images, labels = images.to(DEVICE), labels.to(DEVICE)

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

        val_loss /= len(val_dataset)
        scheduler.step(val_loss)

        # 최적의 검증 손실 업데이트
        best_val_loss = min(best_val_loss, val_loss)

    return best_val_loss

In [13]:
# Optuna 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=OPTUNA_NUM_TRIALS)

# 최적 하이퍼파라미터 출력
print("Best trial:", study.best_trial.params)

[I 2024-11-30 07:59:52,920] A new study created in memory with name: no-name-c8def79e-fabc-4b67-81be-20faff0940f3


[Epoch 1/10]


[Valid]: 100%|██████████| 188/188 [15:43<00:00,  5.02s/it]


[Epoch 2/10]


[Valid]: 100%|██████████| 188/188 [00:30<00:00,  6.21it/s]


...

[Epoch 9/10]


[Valid]: 100%|██████████| 188/188 [00:29<00:00,  6.39it/s]


[Epoch 10/10]


[Valid]: 100%|██████████| 188/188 [00:29<00:00,  6.44it/s]
[I 2024-11-30 14:13:22,358] Trial 9 finished with value: 0.696315263748169 and parameters: {'lr': 2.0099493877764736e-05, 'weight_decay': 2.6054394671479442e-06}. Best is trial 6 with value: 0.6374192107940714.


Best trial: {'lr': 0.00024517227277042787, 'weight_decay': 6.0928726458831747e-05}


In [14]:
with open("optuna_study.pkl", "wb") as f:
    pickle.dump(study, f)

In [30]:
# with open("optuna_study.pkl", "rb") as f:
#     loaded_study = pickle.load(f)

### 4. 최종 학습

In [31]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # tqdm을 배치 루프에 적용
    train_bar = tqdm(train_loader, desc="Training", leave=True)

    for inputs, labels in train_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # Forward pass
        predictions = model(inputs)
        loss = criterion(predictions, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = predictions.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # tqdm 바 업데이트
        train_bar.set_postfix({
            "Loss": f"{running_loss / total:.4f}",
            "Accuracy": f"{100. * correct / total:.2f}%"
        })

    return running_loss / total, 100. * correct / total


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    eval_bar = tqdm(loader, desc="Evaluating", leave=True)

    with torch.no_grad():
        for inputs, labels in eval_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            loss = criterion(predictions, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = predictions.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # tqdm 바 업데이트
            eval_bar.set_postfix({
                "Loss": f"{running_loss / total:.4f}",
                "Accuracy": f"{100. * correct / total:.2f}%"
            })

    return running_loss / total, 100. * correct / total


def trainer(num_epochs, model, train_loader, val_loader, optimizer, scheduler, criterion, device, patience=10):
    # 모델 저장, 조기 종료 조건 설정
    best_val_accuracy = 0.0
    best_val_loss = float('inf')
    counter = 0

    # 히스토리 저장
    train_loss_history, val_loss_history = [], []
    train_acc_history, val_acc_history = [], []

    # 학습 시작
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # 학습
        train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)

        # 검증
        valid_loss, valid_accuracy = evaluate(model, val_loader, criterion, device)

        # 히스토리에 추가
        train_loss_history.append(train_loss)
        val_loss_history.append(valid_loss)
        train_acc_history.append(train_accuracy)
        val_acc_history.append(valid_accuracy)

        print(f"Train Loss: {train_loss:.4f} \tTrain Accuracy: {train_accuracy:.2f}%")
        print(f"Valid Loss: {valid_loss:.4f} \tValid Accuracy: {valid_accuracy:.2f}%")

        # 학습률 업데이트
        scheduler.step(valid_loss)

        # 최고 성능 모델 저장 (validation accuracy 기준)
        if valid_accuracy > best_val_accuracy:
            best_val_accuracy = valid_accuracy
            torch.save(model.state_dict(), "best_model.pth")

        # Early Stopping
        if valid_loss < best_val_loss:
            best_val_loss = valid_loss
            counter = 0  # 개선되면 counter 초기화
        else:
            counter += 1  # 개선되지 않으면 counter 증가

        if counter >= patience:
            print(f"Early stopping after {epoch + 1} epochs")
            break

    print(f"Best Validation Accuracy: {best_val_accuracy:.2f}%")

    return train_loss_history, val_loss_history, train_acc_history, val_acc_history

In [ ]:
# 체크포인트 저장 함수
def save_checkpoint(model, optimizer, scheduler, epoch, path="checkpoint.pth"):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'epoch': epoch
    }, path)
    print(f"Checkpoint saved at epoch {epoch}")

# 체크포인트 로드 함수
def load_checkpoint(model, optimizer, scheduler, path="checkpoint.pth"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler and 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']
    print(f"Checkpoint loaded from epoch {epoch}")
    return epoch

def train_in_batches(total_epochs, step_epochs, model, train_loader, val_loader, optimizer, scheduler, criterion, device, patience=10):
    current_epoch = 0  # 시작 epoch
    for step in range(total_epochs // step_epochs):
        print(f"\n=== Batch {step + 1} ===")

        # 이전 체크포인트 로드
        if step > 0:
            current_epoch = load_checkpoint(model, optimizer, scheduler, "checkpoint.pth")

        # trainer 함수 호출 (step_epochs 만큼 학습)
        train_loss_history, val_loss_history, train_acc_history, val_acc_history = trainer(
            num_epochs=step_epochs,
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            scheduler=scheduler,
            criterion=criterion,
            device=device,
            patience=patience
        )

        # 현재 상태를 체크포인트로 저장
        save_checkpoint(model, optimizer, scheduler, current_epoch + step_epochs, "checkpoint.pth")
        current_epoch += step_epochs

        # Early stopping은 각 step에서 개별적으로 수행되므로 필요 시 종료
        if current_epoch >= total_epochs:
            break

    print("Training completed.")

    return train_loss_history, val_loss_history, train_acc_history, val_acc_history


In [32]:
# Optuna 결과 반영
best_params = study.best_trial.params
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])

In [19]:
%%time
# 실행
train_loss_history, val_loss_history, train_acc_history, val_acc_history = train_in_batches(
    total_epochs=NUM_EPOCHS,
    step_epochs=STEP_EPOCHS,
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=criterion,
    device=DEVICE,
    patience=10
)



=== Batch 1 ===

Epoch 1/10


Evaluating: 100%|██████████| 188/188 [00:32<00:00,  5.82it/s, Loss=0.7061, Accuracy=77.57%]


Train Loss: 1.0399, Train Accuracy: 68.40%
Val Loss: 0.7061, Val Accuracy: 77.57%

Epoch 2/10


Evaluating: 100%|██████████| 188/188 [00:31<00:00,  5.96it/s, Loss=0.6889, Accuracy=79.00%]


Train Loss: 1.0248, Train Accuracy: 69.02%
Val Loss: 0.6889, Val Accuracy: 79.00%

Epoch 3/10


...

Train Loss: 0.9182, Train Accuracy: 71.48%
Val Loss: 0.6411, Val Accuracy: 80.63%

Epoch 7/10


Evaluating: 100%|██████████| 188/188 [00:30<00:00,  6.18it/s, Loss=0.6353, Accuracy=80.97%]


Train Loss: 0.9246, Train Accuracy: 71.63%
Val Loss: 0.6353, Val Accuracy: 80.97%

Epoch 8/10


Training:  35%|███▌      | 198/563 [00:41<01:17,  4.74it/s, Loss=0.8784, Accuracy=73.01%]


KeyboardInterrupt: 

### 5. 학습 결과 확인 및 Test

In [34]:
# 테스트
model.load_state_dict(torch.load("best_model.pth"))
test_loss, test_accuracy = evaluate(model, test_loader, criterion, DEVICE)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Evaluating: 100%|██████████| 188/188 [12:08<00:00,  3.88s/it, Loss=0.6417, Accuracy=80.10%]

Test Loss: 0.6417, Test Accuracy: 80.10%


In [39]:
# 현재 작업 중인 노트북을 HTML로 변환
!jupyter nbconvert --to html --template lab efficientnet_v2_transfer_learning.ipynb

[NbConvertApp] Converting notebook efficientnet_v2_transfer_learning.ipynb to html
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('metadata' was unexpected)

Failed validating 'additionalProperties' in stream:

On instance['cells'][22]['outputs'][0]:
{'metadata': {'tags': None},
 'name': 'stdout',
 'output_type': 'stream',
 'text': '\n=== Batch 1 ===\n\nEpoch 1/10\n'}
[NbConvertApp] Writing 470866 bytes to efficientnet_v2_transfer_learning.html
